In [ ]:
# prototype_development.ipynb

# -------------------------------
# 1. Setup & Install Dependencies
# -------------------------------
!pip install transformers datasets flask -q

import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from datasets import load_dataset
from flask import Flask, request, jsonify
import random

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Running on:", device)

# -------------------------------
# 2. Load Model & Tokenizer
# -------------------------------
model_name = "distilbert-base-multilingual-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name).to(device)

# -------------------------------
# 3. Load Dataset (Crisis-Related Queries)
# -------------------------------
dataset = load_dataset("civil_comments", split="train[:1000]")
comments = dataset["text"]

# Simulating intent labels
intent_labels = ["mental_health", "legal_aid", "domestic_violence", "general_help"]
random.seed(42)

# -------------------------------
# 4. Simple Intent Classification
# -------------------------------
def classify_intent(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128).to(device)
    with torch.no_grad():
        outputs = model(**inputs)
        probs = torch.softmax(outputs.logits, dim=-1)
    intent = intent_labels[torch.argmax(probs)]
    return intent, probs.max().item()

# -------------------------------
# 5. Flask API for Real-Time Queries
# -------------------------------
app = Flask(__name__)

@app.route('/predict', methods=['POST'])
def predict():
    data = request.json
    query = data.get('query')
    
    if query:
        intent, confidence = classify_intent(query)
        response = {
            "intent": intent,
            "confidence": confidence,
            "message": f"Your query was classified as {intent} with {confidence*100:.2f}% confidence."
        }
    else:
        response = {"error": "No query provided"}
    
    return jsonify(response)

# -------------------------------
# 6. Run the Flask App (locally or on Heroku)
# -------------------------------
if __name__ == "__main__":
    app.run(debug=True)

